# Export the configuration

This notebook demonstrates how to save both the Vitessce configuration and the local data it points to in a new folder. For instance, the folder exported via this method can be used when uploading a Vitessce configuration intended for a publication page on the HuBMAP Portal.

In [1]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
)
from os.path import join
import json

In [2]:
vc = VitessceConfig(schema_version="1.0.15", name='Habib et al', description='COVID-19 Healthy Donor Brain')

# Add data
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(
        adata_path=join("processed_data", "habib17.zarr"),
        obs_embedding_paths=["obsm/X_umap"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=["obs/CellType"],
        obs_set_names=["Cell Type"],
        obs_feature_matrix_path="X"
    )
)

# Add views
scatterplot = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
genes = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(vt.HEATMAP, dataset=dataset)
cell_set_sizes = vc.add_view(vt.OBS_SET_SIZES, dataset=dataset)

# Arrange views
vc.layout((scatterplot | cell_sets) / (heatmap | (genes | cell_set_sizes)));

## Define the output folder

We can construct a local path to a folder where the exported configuration and data will be saved.

In [3]:
OUT_DIR = join(".", "exported_data")

## Export the data

The [export](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.export) function can be used to save all local data associated with the configuration. When the parameter `to="files"` is used, the data will be saved to a local folder (specified by `out_dir`).

The `base_url` parameter is also required. It is used to specify the server where the data will ultimately be hosted. For HuBMAP publication pages, this URL is unknown ahead of time, but will ultimately point to something like `http://assets.hubmapconsortium.org/MY_DATASET_ID/`.

For now, please use the `BASE_URL_PLACEHOLDER` variable exported by the `vitessce` package:

In [4]:
from vitessce import BASE_URL_PLACEHOLDER

The `export` function will also return its `VitessceConfig` object as a Python `dict` which can be saved to a JSON file.

In [5]:
config_dict = vc.export(to="files", base_url=BASE_URL_PLACEHOLDER, out_dir=OUT_DIR)

## Save the configuration to a JSON file

To save the `config_dict` variable returned above to a JSON file, we can use the [json.dump](https://docs.python.org/3.8/library/json.html#json.dump) function that comes with Python, along with the [open](https://docs.python.org/3.8/tutorial/inputoutput.html#reading-and-writing-files) context manager:

In [9]:
# Use `open` to create a new empty file at ./exported_data/vitessce.json
with open(join(OUT_DIR, "vitessce.json"), "w") as f:
    json.dump(config_dict, f)

Check out the contents of the new `./exported_data` folder if you are interested.